In [1]:
import tensorflow as tf


c:\users\gebruiker\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.__version__

'1.8.0'

In [7]:
training_dataset = tf.data.Dataset.range(100).map(
    lambda x: x + tf.random_uniform([], -10, 10, tf.int64))
validation_dataset = tf.data.Dataset.range(50)

# Build an iterator that can take different datasets with the same type and shape
iterator = tf.data.Iterator.from_structure(training_dataset.output_types, training_dataset.output_shapes)
next_element = iterator.get_next()

# Get 2 init op for 2 different dataset
training_init_op = iterator.make_initializer(training_dataset)
validation_init_op = iterator.make_initializer(validation_dataset)

with tf.Session() as sess:
    #sess.run(validation_init_op)
    for _ in range(20):
        #sess.run(training_init_op)
        #for _ in range(100):
        #    sess.run(next_element)

        sess.run(validation_init_op)   #very cool
        for _ in range(50):
            sess.run(next_element)

In [17]:
tf.reset_default_graph()

In [24]:
#import tensorflow as tf
#import os

#data_path = 'C:/Users/Gebruiker/PycharmProjects/TFRecords/train.tfrecords'
data_path = './extra.tfrecords'
def parser(serialized_example):
    """Parses a single tf.Example into image and label tensors."""
    feature_myTFrecord = {'train/image': tf.FixedLenFeature([], tf.string), #you wrote the files even in the validate set as train BAKAYARO
               'train/label': tf.FixedLenFeature([], tf.int64)}
    
    features = tf.parse_single_example(
        serialized_example,features= feature_myTFrecord)
    
    
    image = tf.decode_raw(features['train/image'], tf.float32)

    # Cast label data into int32
    temp_label = tf.cast(features['train/label'], tf.int32)
    # Reshape image data into the original shape
    image = tf.reshape(image, [575, 400, 1])

    label = tf.cast(tf.subtract(temp_label, 1), tf.int32)

    return image, label


#filenames = [os.path.join(".", 'f1.tfrecords'), os.path.join(".", 'f2.tfrecords')]
filenames = data_path
batch_size =3 
NUM_PER_EPOCH = 30
# Repeat infinitely.
dataset = tf.data.TFRecordDataset(filenames,buffer_size=2 * batch_size,num_parallel_reads=4)#.repeat()

# Parse records.
dataset = dataset.map(parser)

# Potentially shuffle records.
min_queue_examples = int(NUM_PER_EPOCH * 0.4)
dataset = dataset.shuffle(buffer_size=min_queue_examples + 3 * batch_size)
# Batch it up.
dataset = dataset.batch(batch_size)
    #iterator = dataset.make_one_shot_iterator()
    #image_batch, label_batch = iterator.get_next()
iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
next_element = iterator.get_next()    

training_init_op = iterator.make_initializer(dataset)

#return image_batch, label_batch

In [25]:
#sess.run(training_init_op)
        #for _ in range(100):
        #    sess.run(next_element)

In [26]:
n_epochs = 20
counts = 0
with tf.Session() as sess:
    sess.run(training_init_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess,coord=coord)
    for epoch in range(n_epochs):
        for iteration in range(10):
            print(epoch,' epoch and iteration ' , iteration)
            try:
                X_batch, y_batch = sess.run(next_element)
                counts = counts + 1
            except tf.errors.OutOfRangeError:
                print('error has occured!!! ' , iteration)
                sess.run(training_init_op)
                break
            
    coord.request_stop()

    # Wait for threads to stop
    coord.join(threads)
print(counts)

0  epoch and iteration  0
0  epoch and iteration  1
0  epoch and iteration  2
0  epoch and iteration  3
0  epoch and iteration  4
0  epoch and iteration  5
0  epoch and iteration  6
0  epoch and iteration  7
0  epoch and iteration  8
0  epoch and iteration  9
1  epoch and iteration  0
1  epoch and iteration  1
1  epoch and iteration  2
1  epoch and iteration  3
1  epoch and iteration  4
1  epoch and iteration  5
1  epoch and iteration  6
1  epoch and iteration  7
1  epoch and iteration  8
1  epoch and iteration  9
2  epoch and iteration  0
2  epoch and iteration  1
2  epoch and iteration  2
2  epoch and iteration  3
2  epoch and iteration  4
2  epoch and iteration  5
2  epoch and iteration  6
2  epoch and iteration  7
2  epoch and iteration  8
2  epoch and iteration  9
3  epoch and iteration  0
error has occured!!!  0
4  epoch and iteration  0
4  epoch and iteration  1
4  epoch and iteration  2
4  epoch and iteration  3
4  epoch and iteration  4
4  epoch and iteration  5
4  epoch and i

In [29]:
import numpy as np

In [30]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [36]:
reset_graph()

In [37]:
data_path = './extra.tfrecords'
def parser(serialized_example):
    """Parses a single tf.Example into image and label tensors."""
    feature_myTFrecord = {'train/image': tf.FixedLenFeature([], tf.string), #you wrote the files even in the validate set as train BAKAYARO
               'train/label': tf.FixedLenFeature([], tf.int64)}
    
    features = tf.parse_single_example(
        serialized_example,features= feature_myTFrecord)
    
    
    image = tf.decode_raw(features['train/image'], tf.float32)

    # Cast label data into int32
    temp_label = tf.cast(features['train/label'], tf.int32)
    # Reshape image data into the original shape
    image = tf.reshape(image, [575, 400, 1])

    label = tf.cast(tf.subtract(temp_label, 1), tf.int32)

    return image, label


#filenames = [os.path.join(".", 'f1.tfrecords'), os.path.join(".", 'f2.tfrecords')]
filenames = data_path
batch_size =3 
NUM_PER_EPOCH = 30
# Repeat infinitely.
dataset = tf.data.TFRecordDataset(filenames,buffer_size=2 * batch_size,num_parallel_reads=4)#.repeat()

# Parse records.
dataset = dataset.map(parser)

# Potentially shuffle records.
min_queue_examples = int(NUM_PER_EPOCH * 0.4)
dataset = dataset.shuffle(buffer_size=min_queue_examples + 3 * batch_size)
# Batch it up.
dataset = dataset.batch(batch_size)
    #iterator = dataset.make_one_shot_iterator()
    #image_batch, label_batch = iterator.get_next()
iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
next_element = iterator.get_next()    

training_init_op = iterator.make_initializer(dataset)

In [38]:
height = 575
width = 400
channels = 1
n_inputs = height * width

conv1_fmaps = 96
conv1_ksize = 3
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 128
conv2_ksize = 3
conv2_stride = 2
conv2_pad = "SAME"

pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = 36


with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, height,width,channels], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None],name = 'y')

    ####to get the labels start from 0
    

conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, name="conv1")
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv2")

with tf.name_scope("pool3"):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 144 * 100])

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.relu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()


In [39]:
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

In [40]:

n_epochs = 10
#n_epochs = 5
with tf.Session() as sess:
    sess.run(init_op)
    # Create a coordinator and run all QueueRunner objects
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess,coord=coord)
    sess.run(training_init_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess,coord=coord)
    for epoch in range(n_epochs):
        for iteration in range(10):
            print(epoch,' epoch and iteration ' , iteration)
            try:
                X_batch, y_batch = sess.run(next_element)
            except tf.errors.OutOfRangeError:
                print('error has occured!!! ' , iteration)
                sess.run(training_init_op)
                X_batch, y_batch = sess.run(next_element)
                break
            sess.run([training_op], feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        print(epoch, "Train accuracy:", acc_train)
    
    coord.request_stop()

    # Wait for threads to stop
    coord.join(threads)


0  epoch and iteration  0
0  epoch and iteration  1
0  epoch and iteration  2
0  epoch and iteration  3
0  epoch and iteration  4
0  epoch and iteration  5
0  epoch and iteration  6
0  epoch and iteration  7
0  epoch and iteration  8
0  epoch and iteration  9
0 Train accuracy: 0.33333334
1  epoch and iteration  0
1  epoch and iteration  1
1  epoch and iteration  2
1  epoch and iteration  3
1  epoch and iteration  4
1  epoch and iteration  5
1  epoch and iteration  6
1  epoch and iteration  7
1  epoch and iteration  8
1  epoch and iteration  9
1 Train accuracy: 1.0
2  epoch and iteration  0
2  epoch and iteration  1
2  epoch and iteration  2
2  epoch and iteration  3
2  epoch and iteration  4
2  epoch and iteration  5
2  epoch and iteration  6
2  epoch and iteration  7
2  epoch and iteration  8
2  epoch and iteration  9
2 Train accuracy: 1.0
3  epoch and iteration  0
error has occured!!!  0
3 Train accuracy: 1.0
4  epoch and iteration  0
4  epoch and iteration  1
4  epoch and iteration 